In [1]:
import copy

import pandas as pd
from nltk.corpus import floresta as flt
from nltk.corpus import machado as mch
from nltk.corpus import mac_morpho as mcm

from utils import load_six_emotions, load_valence_emotions_from_oplexicon, load_valence_emotions_from_sentilex, generate_corpus
from unsupervised.emotional_lsa import EmotionalLSA

In [5]:
with open('/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/frases.txt') as h:
    original_phrases = h.readlines()
original_phrases = original_phrases[:25]
phrases = copy.deepcopy(original_phrases)
# phrases += [' '.join(f).replace('_', ' ') for f in flt.sents()[:2000]]
# phrases += [' '.join(f).replace('_', ' ') for f in mch.sents()[:2000]]
# phrases += [' '.join(f).replace('_', ' ') for f in mcm.sents()[:2000]]

# original_phrases = []
# sents = []
# filename = '/home/rdenadai/programas/vagrant_img/vagrant.machine.puphpet/projetos/github/sentiment-analysis-2018-president-election/dataset/tweets-tratados-polaridade-10-10-2018.csv'
# with open(filename, 'r') as h:
#     tweets = h.readlines()
#     for tweet in tweets[1:70]:
#         rep = ['\n', '"', '@']
#         for r in rep:
#             tweet = tweet.replace(r, '')
#         infos = tweet.split('|')
#         print(infos)
#         original_phrases += [infos[1].strip()]
#         sents += [infos[2].strip()]

# phrases = copy.deepcopy(original_phrases)
# print(phrases)
# print(sents)
phrases = generate_corpus(phrases)
ldocs = [f'D{i}' for i in range(len(original_phrases))]

Iniciando processamento...
Filtro e finalização...
Finalizado...


In [6]:
print('Loading emotional words: ')
emotion_words = load_six_emotions('/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/emocoes')
emotion_words_n = load_valence_emotions_from_oplexicon('/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/emocoes/oplexicon_v3.0/lexico_v3.0.txt')
emotion_words['NEUTRO'] = emotion_words_n['NEUTRO']

print('Starting EmotionalLSA model...')
model = EmotionalLSA(debug=True)
print('fit and transform...')
predicted = model.fit_transform(phrases, emotion_words)

Loading emotional words: 
Starting EmotionalLSA model...
fit and transform...
Count:
Actual number of features: 119
Processing emotional state... this may take a while...


In [10]:
df = pd.DataFrame(predicted[:len(original_phrases)], index=ldocs, columns=emotion_words.keys())
for i, frase in enumerate(original_phrases):
    print('-' * 20)
    print(f'D{i} - {frase}')
    print(df.loc[f'D{i}'])

--------------------
D0 - "A alma jamais pode vaguear sem rumo, se tomar a Bíblia para lhe guiar os passos."

ALEGRIA     -3.93
DESGOSTO    17.73
MEDO       -19.26
RAIVA       61.92
SURPRESA    17.73
TRISTEZA    17.17
NEUTRO       7.36
Name: D0, dtype: float64
--------------------
D1 - "A religiao é o que impede o pobre de matar o rico.

ALEGRIA     23.22
DESGOSTO    23.22
MEDO        23.22
RAIVA       23.22
SURPRESA    23.22
TRISTEZA    23.22
NEUTRO      23.22
Name: D1, dtype: float64
--------------------
D2 - "Circunstâncias? O que são as circunstâncias? Eu sou as circunstâncias!"

ALEGRIA      14.46
DESGOSTO    100.00
MEDO         -5.94
RAIVA        17.01
SURPRESA    100.00
TRISTEZA     21.60
NEUTRO       26.76
Name: D2, dtype: float64
--------------------
D3 - "Que felicidade a Bíblia proporciona àqueles que acreditam nela! Que maravilhas admiram aqueles que refletem nela! "

ALEGRIA      73.40
DESGOSTO     23.47
MEDO        -38.92
RAIVA       100.00
SURPRESA     23.47
TRISTEZA    

In [13]:
print('Loading emotional words: ')
emotion_words = load_valence_emotions_from_oplexicon('/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/emocoes/oplexicon_v3.0/lexico_v3.0.txt')

print('Starting EmotionalLSA model...')
model = EmotionalLSA(debug=True)
print('fit and transform...')
predicted = model.fit_transform(phrases, emotion_words)
    
size = len(original_phrases)
df = pd.DataFrame(predicted[:size], index=ldocs, columns=emotion_words.keys())
for i, frase in enumerate(original_phrases):
    print('-' * 20)
    print(f'D{i} - {frase}')
    print(df.loc[f'D{i}'])

Loading emotional words: 
Starting EmotionalLSA model...
fit and transform...
Count:
Actual number of features: 119
Processing emotional state... this may take a while...
--------------------
D0 - "A alma jamais pode vaguear sem rumo, se tomar a Bíblia para lhe guiar os passos."

POSITIVO    66.36
NEGATIVO    -8.63
NEUTRO      41.07
Name: D0, dtype: float64
--------------------
D1 - "A religiao é o que impede o pobre de matar o rico.

POSITIVO    7.05
NEGATIVO    7.05
NEUTRO      7.05
Name: D1, dtype: float64
--------------------
D2 - "Circunstâncias? O que são as circunstâncias? Eu sou as circunstâncias!"

POSITIVO    74.35
NEGATIVO    65.25
NEUTRO      80.96
Name: D2, dtype: float64
--------------------
D3 - "Que felicidade a Bíblia proporciona àqueles que acreditam nela! Que maravilhas admiram aqueles que refletem nela! "

POSITIVO    24.86
NEGATIVO    44.14
NEUTRO      55.38
Name: D3, dtype: float64
--------------------
D4 - A adolescência é como uma casa em dia de mudanças: Uma de

In [14]:
print('Loading emotional words: ')
emotion_words = load_valence_emotions_from_sentilex(
        '/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/emocoes/SentiLex-PT02/SentiLex-flex-PT02.txt')

print('Starting EmotionalLSA model...')
model = EmotionalLSA(debug=True)
print('fit and transform...')
predicted = model.fit_transform(phrases, emotion_words)

size = len(original_phrases)
df = pd.DataFrame(predicted[:size], index=ldocs, columns=emotion_words.keys())
for i, frase in enumerate(original_phrases):
    print('-' * 20)
    print(f'D{i} - {frase}')
    print(df.loc[f'D{i}'])

Loading emotional words: 
Starting EmotionalLSA model...
fit and transform...
Count:
Actual number of features: 119
Processing emotional state... this may take a while...
--------------------
D0 - "A alma jamais pode vaguear sem rumo, se tomar a Bíblia para lhe guiar os passos."

POSITIVO    47.39
NEGATIVO    31.21
NEUTRO     -20.74
Name: D0, dtype: float64
--------------------
D1 - "A religiao é o que impede o pobre de matar o rico.

POSITIVO    2.14
NEGATIVO    2.14
NEUTRO      2.14
Name: D1, dtype: float64
--------------------
D2 - "Circunstâncias? O que são as circunstâncias? Eu sou as circunstâncias!"

POSITIVO    27.59
NEGATIVO    60.21
NEUTRO     -15.27
Name: D2, dtype: float64
--------------------
D3 - "Que felicidade a Bíblia proporciona àqueles que acreditam nela! Que maravilhas admiram aqueles que refletem nela! "

POSITIVO    15.41
NEGATIVO    29.34
NEUTRO     -11.68
Name: D3, dtype: float64
--------------------
D4 - A adolescência é como uma casa em dia de mudanças: Uma de